# Notebook 05: Advanced Experiments

This notebook explores four advanced topics at the frontier of retrocausality research:

1. **GHZ States & Mermin Inequality** — All-or-nothing nonlocality
2. **Decoherence** — How noise destroys quantum correlations
3. **Phase Transition Model** — Collapse as a phase transition (Tlalpan Interpretation)
4. **Quantum Speedup as Retrocausality** — Castagnoli's thesis

In [ ]:
import sys
sys.path.insert(0, '..')

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

---

## Part 1: GHZ States and the Mermin Inequality

**Reference**: Greenberger, Horne, Zeilinger (1989); Mermin (1990)

The 3-qubit GHZ state $|GHZ\rangle = (|000\rangle + |111\rangle)/\sqrt{2}$ shows **all-or-nothing** nonlocality: the Mermin value reaches the algebraic maximum of 4, versus the classical bound of 2.

In [ ]:
from src.advanced.multipartite import ghz_mermin_test, w_state_entanglement_analysis

result = ghz_mermin_test()

print("GHZ-Mermin Inequality Test")
print("=" * 55)
print(f"  |Mermin value| = {abs(result['mermin_value']):.4f}")
print(f"  Classical bound = {result['classical_bound']:.1f}")
print(f"  QM prediction = {result['qm_prediction']:.1f}")
print(f"  Violates: {result['violates']}")
print(f"\nIndividual correlators:")
for name, val in result['correlators'].items():
    print(f"  {name} = {val:+.4f}")
print(f"\n{result['explanation']}")

In [ ]:
# Visualize GHZ vs Classical bound
fig, ax = plt.subplots(figsize=(8, 5))

models = ['Classical\nBound', 'GHZ State\n(QM)']
values = [result['classical_bound'], abs(result['mermin_value'])]
colors = ['#FF9800', '#2196F3']

bars = ax.bar(models, values, color=colors, edgecolor='black', linewidth=1.5, width=0.5)
ax.axhline(y=2, color='red', linestyle='--', linewidth=2, label='Classical bound = 2')
ax.axhline(y=4, color='blue', linestyle=':', linewidth=2, label='Algebraic maximum = 4')

for bar, val in zip(bars, values):
    ax.text(bar.get_x() + bar.get_width()/2., bar.get_height() + 0.1,
            f'{val:.2f}', ha='center', fontsize=14, fontweight='bold')

ax.set_ylabel('|Mermin Value|', fontsize=13)
ax.set_title('GHZ State: All-or-Nothing Violation of Mermin Inequality', fontsize=13)
ax.set_ylim(0, 5)
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3, axis='y')
plt.tight_layout()
plt.show()

In [ ]:
# W state entanglement analysis
w_result = w_state_entanglement_analysis(3)

print("\nW State Entanglement Analysis")
print("=" * 55)
print(f"  Robust against particle loss: {w_result['w_state_robust']}")
for key, info in w_result['subsystem_analysis'].items():
    print(f"\n  {key}:")
    print(f"    Purity = {info['purity']:.4f}")
    print(f"    Entropy = {info['entropy']:.4f}")
    print(f"    Still entangled: {info['still_entangled']}")
print(f"\n{w_result['explanation']}")

---

## Part 2: Decoherence — How Noise Destroys Quantum Correlations

Real experiments are noisy. As decoherence increases, Bell violations disappear. The **critical noise threshold** is where $|S| = 2$ — below this, no Bell violation is possible.

In [ ]:
from src.advanced.decoherence import (
    bell_violation_vs_noise, entanglement_vs_noise, concurrence
)
from src.core.states import bell_state
from src.core.density_matrix import state_to_density

# CHSH value vs noise
df_depol = bell_violation_vs_noise('depolarizing', n_points=50)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

ax1.plot(df_depol['noise_level'], np.abs(df_depol['chsh_value']), 'b-', linewidth=2)
ax1.axhline(y=2, color='red', linestyle='--', linewidth=2, label='Classical bound |S|=2')
ax1.axhline(y=2*np.sqrt(2), color='green', linestyle=':', label=f'Tsirelson bound {2*np.sqrt(2):.3f}')
ax1.fill_between(df_depol['noise_level'], 0, 2, alpha=0.1, color='red', label='No violation zone')

# Find critical threshold
critical_idx = np.argmin(np.abs(np.abs(df_depol['chsh_value'].values) - 2.0))
p_critical = df_depol['noise_level'].iloc[critical_idx]
ax1.axvline(x=p_critical, color='orange', linestyle='-.', label=f'Critical p ~ {p_critical:.3f}')

ax1.set_xlabel('Depolarizing noise level p', fontsize=12)
ax1.set_ylabel('|CHSH value S|', fontsize=12)
ax1.set_title('Bell Violation vs Depolarizing Noise', fontsize=13)
ax1.legend(fontsize=9)
ax1.grid(True, alpha=0.3)

# Concurrence vs noise
df_ent = entanglement_vs_noise('depolarizing', n_points=50)

ax2.plot(df_ent['noise_level'], df_ent['concurrence'], 'purple', linewidth=2, label='Concurrence')
ax2.plot(df_ent['noise_level'], df_ent['subsystem_entropy'], 'orange', linewidth=2, label='Subsystem entropy')
ax2.set_xlabel('Depolarizing noise level p', fontsize=12)
ax2.set_ylabel('Value', fontsize=12)
ax2.set_title('Entanglement Death Under Noise', fontsize=13)
ax2.legend(fontsize=11)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Critical noise threshold for Bell violation: p ~ {p_critical:.3f}")
print(f"(Theory for depolarizing: p = 1 - 1/sqrt(2) ~ {1 - 1/np.sqrt(2):.3f})")

---

## Part 3: Collapse as Phase Transition (Tlalpan Interpretation)

**Reference**: "Time Symmetry, Retrocausality, and Emergent Collapse" (2025)

The Tlalpan Interpretation proposes that wavefunction collapse is a **phase transition**: as the measurement coupling strength increases, the system undergoes a sharp transition from quantum superposition to classical definiteness.

In [ ]:
from src.advanced.phase_transition import (
    interference_visibility_vs_amplification,
    finite_size_scaling
)

# Coherence vs coupling strength for different amplification stages
df_vis = interference_visibility_vs_amplification(
    coupling_range=np.linspace(0.01, 2.0, 40),
    n_stages_list=[1, 2, 3, 5],
    apparatus_dim=30
)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

for n_stages in [1, 2, 3, 5]:
    data = df_vis[df_vis['n_stages'] == n_stages]
    ax1.plot(data['coupling'], data['coherence'], '-o', markersize=3,
             label=f'{n_stages} stage{"s" if n_stages > 1 else ""}')

ax1.set_xlabel('Coupling strength g', fontsize=12)
ax1.set_ylabel('Coherence |rho_01|', fontsize=12)
ax1.set_title('Coherence vs Coupling: Phase Transition?', fontsize=13)
ax1.legend(fontsize=10)
ax1.grid(True, alpha=0.3)

# Finite-size scaling
df_fss = finite_size_scaling(
    apparatus_dims=[10, 20, 40],
    coupling_range=np.linspace(0.01, 2.0, 30)
)

for dim in [10, 20, 40]:
    data = df_fss[df_fss['apparatus_dim'] == dim]
    ax2.plot(data['coupling'], data['coherence'], '-o', markersize=3,
             label=f'apparatus dim = {dim}')

ax2.set_xlabel('Coupling strength g', fontsize=12)
ax2.set_ylabel('Coherence |rho_01|', fontsize=12)
ax2.set_title('Finite-Size Scaling: Does the Transition Sharpen?', fontsize=13)
ax2.legend(fontsize=10)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Observation: As apparatus dimension increases, the coherence drop")
print("becomes sharper, consistent with a phase-transition picture.")
print("More amplification stages accelerate the transition.")

---

## Part 4: Quantum Speedup as Retrocausality

**Reference**: Castagnoli (2025), arXiv:2505.08346

Castagnoli's thesis: quantum computational speedup can be understood as retrocausality. In the TSVF framework, quantum algorithms appear to be "guided toward" the correct answer by the future post-selection.

In [ ]:
from src.advanced.speedup_analysis import (
    deutsch_jozsa_tsvf_analysis, grover_tsvf_analysis
)

# Deutsch-Jozsa analysis
print("DEUTSCH-JOZSA ALGORITHM — TSVF Analysis")
print("=" * 60)

for oracle in ['constant_0', 'constant_1', 'balanced_id', 'balanced_not']:
    result = deutsch_jozsa_tsvf_analysis(oracle)
    wv = result['weak_value_proj1_intermediate']
    wv_str = f"{wv.real:+.4f}" if not np.isnan(wv.real) else "N/A"
    print(f"\n  Oracle: {oracle}")
    print(f"    Result: {result['result']} (correct: {result['correct']})")
    print(f"    P(constant) = {result['p_constant']:.4f}, P(balanced) = {result['p_balanced']:.4f}")
    print(f"    Weak value of |1><1| at intermediate step: {wv_str}")

In [ ]:
# Grover's algorithm TSVF analysis
print("\nGROVER'S ALGORITHM — TSVF Step-by-Step Analysis")
print("=" * 60)

marked_items = [0, 1, 2, 3]
fig, axes = plt.subplots(1, len(marked_items), figsize=(16, 4))

for idx, item in enumerate(marked_items):
    result = grover_tsvf_analysis(marked_item=item)
    steps = result['steps']
    
    step_names = list(steps.keys())
    born_probs = [steps[s]['born_probability'] for s in step_names]
    wv_reals = []
    for s in step_names:
        wv = steps[s]['weak_value_marked']
        wv_reals.append(wv.real if not np.isnan(wv.real) else 0)
    
    x = np.arange(len(step_names))
    width = 0.35
    axes[idx].bar(x - width/2, born_probs, width, label='Born P(marked)', color='steelblue')
    axes[idx].bar(x + width/2, wv_reals, width, label='Weak value', color='orange')
    axes[idx].set_xticks(x)
    axes[idx].set_xticklabels(['Init', 'Oracle', 'Final'], fontsize=9)
    axes[idx].set_title(f'Item {item}\nP(find)={result["final_probability"]:.2f}', fontsize=10)
    axes[idx].set_ylim(0, 1.2)
    if idx == 0:
        axes[idx].set_ylabel('Value')
        axes[idx].legend(fontsize=8)
    axes[idx].grid(True, alpha=0.3, axis='y')

plt.suptitle('Grover Search: Born Probability vs Weak Value at Each Step',
             fontsize=13, fontweight='bold')
plt.tight_layout()
plt.show()

print("\nThe weak value of the marked-item projector shows how the system")
print("is 'drawn toward' the correct answer at each step.")
print("In the retrocausal interpretation, the future post-selection (finding")
print("the marked item) retroactively guides the intermediate evolution.")

---

## Summary: The Boundary Between Quantum and Classical

| Experiment | Key Finding | Retrocausal Implication |
|---|---|---|
| GHZ-Mermin | Perfect violation (M=4) | Retrocausal model must explain all-or-nothing nonlocality |
| Decoherence | Critical noise threshold | Retrocausal correlations are fragile — noise destroys them |
| Phase transition | Sharp coherence drop | Collapse may be a genuine phase transition, not just decoherence |
| Grover/DJ | Weak values anticipate answer | Quantum speedup = retrocausality (Castagnoli thesis) |

### The Grand Picture

Quantum mechanics has **retrocausal correlations** — they are real, measurable, and computationally useful. But they are bounded by the **no-signaling theorem**: you can never use them to send information backward in time.

This boundary is what our project precisely maps out:
- **Notebook 01**: The quantum eraser is NOT time travel
- **Notebook 02**: The no-signaling theorem is mathematically exact
- **Notebook 03**: TSVF reveals the retrocausal structure of quantum mechanics
- **Notebook 04**: Retrocausal models violate Bell while respecting no-signaling
- **Notebook 05**: Advanced implications (this notebook)

The world doesn't need another "quantum = magic" narrative. It needs a **precise computational framework** showing where the boundary lies — and that's what this project provides.